In [1]:
#dependencies 

import numpy as np
import pandas as pd

In [2]:
#import cleaned data csv
all_wells = pd.read_csv('Cleaned_Data/all_wells.csv')
all_wells

,Unnamed: 0,DATEPRD,WELL_BORE_CODE,AVG_DOWNHOLE_PRESSURE,AVG_DOWNHOLE_TEMPERATURE,AVG_DP_TUBING,AVG_ANNULUS_PRESS,AVG_CHOKE_SIZE_P,AVG_WHP_P,AVG_WHT_P,DP_CHOKE_SIZE,BORE_OIL_VOL,BORE_GAS_VOL,BORE_WAT_VOL
0,8027,16-Jan-14,NO 15/9-F-15 D,241.771,104.899,185.90,19.06,8.66,55.87,16.90,27.97,351.05,52394.90,0.00
1,8028,17-Jan-14,NO 15/9-F-15 D,213.852,106.293,162.21,19.87,9.47,51.64,22.36,22.35,485.34,72058.82,0.00
2,8029,18-Jan-14,NO 15/9-F-15 D,206.615,106.375,157.93,27.81,9.69,48.69,22.23,19.86,513.12,77600.88,0.00
3,8030,19-Jan-14,NO 15/9-F-15 D,203.075,106.392,156.51,25.72,9.73,46.57,20.96,18.04,497.55,72928.16,0.00
4,8031,20-Jan-14,NO 15/9-F-15 D,203.981,106.317,157.11,12.03,9.40,46.87,21.19,18.36,469.59,67199.15,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7356,726,02-Apr-16,NO 15/9-F-1 C,223.012,108.058,196.63,NaN,54.69,26.38,69.56,10.68,284.72,46880.14,766.23
7357,727,03-Apr-16,NO 15/9-F-1 C,221.813,108.044,196.13,NaN,54.72,25.68,72.50,9.91,280.17,45842.82,748.25
7358,728,04-Apr-16,NO 15/9-F-1 C,220.780,108.042,195.66,NaN,55.07,25.11,71.75,9.20,281.93,45800.83,797.30
7359,729,05-Apr-16,NO 15/9-F-1 C,218.752,108.078,193.41,NaN,56.00,25.34,72.22,9.10,317.38,51990.65,792.76


# Neural Network Analysis

### AVG_CHOKE_SIZE_P, AVG_WHP_P, AVG_WHT_P, DP_CHOKE_SIZE, BORE_OIL_VOL, BORE_GAS_VOL, BORE_WAT_VOL

In [3]:
#read in data for analysis 
X1= all_wells[["AVG_CHOKE_SIZE_P","AVG_WHP_P","AVG_WHT_P", "DP_CHOKE_SIZE","BORE_OIL_VOL","BORE_GAS_VOL", "BORE_WAT_VOL"]]
y1= all_wells["AVG_DOWNHOLE_PRESSURE"].values.reshape(-1, 1)
print(X1.shape, y1.shape)

(7361, 7) (7361, 1)


In [4]:
#split into test and train data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X1, y1, random_state=42)

In [5]:
from sklearn.preprocessing import StandardScaler
# # Create a StandardScater model and fit it to the training data
X_scaler = StandardScaler().fit(X_train)
y_scaler = StandardScaler().fit(y_train)
# # Transform the training and testing data using the X_scaler and y_scaler models
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
y_train_scaled = y_scaler.transform(y_train)
y_test_scaled = y_scaler.transform(y_test)

In [6]:
#create the neural network
from tensorflow.keras.models import Sequential
model = Sequential()

In [7]:
#create the model
from tensorflow.keras.layers import Dense
number_inputs = X_train.shape[1]
number_hidden_nodes = 100

model.add(Dense(units=number_hidden_nodes,
                activation='relu', input_dim=number_inputs))
model.add(Dense(number_hidden_nodes, kernel_initializer='normal',activation='relu'))
model.add(Dense(1, kernel_initializer='normal',activation='linear'))

In [8]:
#metrics
# CREDIT: https://github.com/keras-team/keras/issues/7947
# root mean squared error (rmse) for regression (only for Keras tensors)
def rmse(y_true, y_pred):
    from keras import backend
    return backend.sqrt(backend.mean(backend.square(y_pred - y_true), axis=-1))

# mean squared error (mse) for regression  (only for Keras tensors)
def mse(y_true, y_pred):
    from keras import backend
    return backend.mean(backend.square(y_pred - y_true), axis=-1)

# coefficient of determination (R^2) for regression  (only for Keras tensors)
def r_square(y_true, y_pred):
    from keras import backend as K
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [9]:
#compile the model
from keras import losses

model.compile(loss="mean_absolute_error",
              optimizer="adam", metrics=[r_square, rmse])

Using TensorFlow backend.


In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               800       
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 11,001
Trainable params: 11,001
Non-trainable params: 0
_________________________________________________________________


In [11]:
from keras.callbacks import EarlyStopping
es= EarlyStopping(monitor='val_loss', min_delta=0.001, patience=5,verbose=0, mode='min')
model.fit(
    X_test_scaled,
    y_test_scaled,
    epochs=100,
    shuffle=True,
    verbose=2,
    validation_split= .15,
    callbacks= [es]
)

Train on 1564 samples, validate on 277 samples
Epoch 1/100
1564/1564 - 0s - loss: 0.6740 - r_square: -5.2061e-02 - rmse: 0.6740 - val_loss: 0.6118 - val_r_square: -1.1185e-01 - val_rmse: 0.6118
Epoch 2/100
1564/1564 - 0s - loss: 0.5659 - r_square: 0.0496 - rmse: 0.5659 - val_loss: 0.5067 - val_r_square: 0.1984 - val_rmse: 0.5067
Epoch 3/100
1564/1564 - 0s - loss: 0.4264 - r_square: 0.3857 - rmse: 0.4264 - val_loss: 0.4000 - val_r_square: 0.3936 - val_rmse: 0.4000
Epoch 4/100
1564/1564 - 0s - loss: 0.3610 - r_square: 0.5132 - rmse: 0.3610 - val_loss: 0.3510 - val_r_square: 0.5237 - val_rmse: 0.3510
Epoch 5/100
1564/1564 - 0s - loss: 0.3294 - r_square: 0.5709 - rmse: 0.3294 - val_loss: 0.3531 - val_r_square: 0.5661 - val_rmse: 0.3531
Epoch 6/100
1564/1564 - 0s - loss: 0.3076 - r_square: 0.6308 - rmse: 0.3076 - val_loss: 0.3034 - val_r_square: 0.6406 - val_rmse: 0.3034
Epoch 7/100
1564/1564 - 0s - loss: 0.2855 - r_square: 0.6756 - rmse: 0.2855 - val_loss: 0.2851 - val_r_square: 0.6757 - v

1564/1564 - 0s - loss: 0.1052 - r_square: 0.8967 - rmse: 0.1052 - val_loss: 0.1084 - val_r_square: 0.8925 - val_rmse: 0.1084


In [12]:
model.evaluate(X_test_scaled, y_test_scaled, batch_size=50)

1841/1841 [==============================] - 0s 15us/sample - loss: 0.1042 - r_square: 0.9001 - rmse: 0.1042


[0.10421198166579024, 0.9001302, 0.10421198]

# Hyperas


In [13]:
#split into test and train data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X1, y1, random_state=42)

In [14]:
# from sklearn.preprocessing import StandardScaler
# # # Create a StandardScater model and fit it to the training data
# X_scaler = StandardScaler().fit(X_train)
# y_scaler = StandardScaler().fit(y_train)
# # # Transform the training and testing data using the X_scaler and y_scaler models
# X_train_scaled = X_scaler.transform(X_train)
# X_test_scaled = X_scaler.transform(X_test)
# y_train_scaled = y_scaler.transform(y_train)
# y_test_scaled = y_scaler.transform(y_test)

In [15]:
def data():
    #read in data for analysis 
    all_wells = pd.read_csv('Cleaned_Data/all_wells.csv')
    X1= all_wells[["AVG_CHOKE_SIZE_P","AVG_WHP_P","AVG_WHT_P", "DP_CHOKE_SIZE","BORE_OIL_VOL","BORE_GAS_VOL", "BORE_WAT_VOL"]]
    y1= all_wells["AVG_DOWNHOLE_PRESSURE"].values.reshape(-1, 1)
    #split into test and train data
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X1, y1, random_state=42)
    from sklearn.preprocessing import StandardScaler
    # # Create a StandardScater model and fit it to the training data
    X_scaler = StandardScaler().fit(X_train)
    y_scaler = StandardScaler().fit(y_train)
    # # Transform the training and testing data using the X_scaler and y_scaler models
    X_train_scaled = X_scaler.transform(X_train)
    X_test_scaled = X_scaler.transform(X_test)
    y_train_scaled = y_scaler.transform(y_train)
    y_test_scaled = y_scaler.transform(y_test)
    
    x_train = X_train_scaled,
    x_test = X_test_scaled, 
    y_train = y_train_scaled,
    y_test = y_test_scaled
    return x_train, y_train, x_test, y_test

In [16]:
#metrics
# CREDIT: https://github.com/keras-team/keras/issues/7947
# root mean squared error (rmse) for regression (only for Keras tensors)
def rmse(y_true, y_pred):
    from keras import backend
    return backend.sqrt(backend.mean(backend.square(y_pred - y_true), axis=-1))

# mean squared error (mse) for regression  (only for Keras tensors)
def mse(y_true, y_pred):
    from keras import backend
    return backend.mean(backend.square(y_pred - y_true), axis=-1)

# coefficient of determination (R^2) for regression  (only for Keras tensors)
def r_square(y_true, y_pred):
    from keras import backend as K
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [17]:
# from __future__ import print_function
# from __future__ import absolute_import, division, print_function
exec('from __future__ import absolute_import, division, print_function')
from hyperas.distributions import uniform
from hyperopt import Trials, STATUS_OK, tpe
from keras.datasets import mnist
from keras.layers.core import Dense, Dropout, Activation
from keras.models import Sequential
from keras.utils import np_utils
from hyperas import optim
from hyperas.distributions import choice, uniform

def create_model(x_train, y_train, x_test, y_test):
    model= Sequential() 
    model.add(Dense({{choice([50,100,150])}}), input_shape=X_train.shape[1])
    model.add(Activation('linear'))
    model.add(Dropout({{choice([.1,.2,.3,.4])}}))
    model.add(Dense({{choice([50,100,150])}}))
    model.add(Activation('linear'))
    model.add(Droput({{choice[.1,.2,.3,.4]}}))
    model.add(Dense(1))
    model.add(Activation('linear'))
#     return model
    model.compile(loss='mean_absolute_error', optimizer= 'adam', metrics=[r_square, rmse])
    result= model.fit(x_train, y_train,
                      batch_size={{choice([64,128])}},
                      epochs={{choice([50,100,150])}},
                      verbose=2,
                      validation_split =0.15)
    validation_acc= np.min(result.history['val_loss'])
    print('Lowest Validation Loss:', validation_acc)
    return {'loss': -accuracy, 'status': STATUS_OK, 'model': model}   

best_run, best_model= optim.minimize(model=create_model,
                                     data=data,
                                     algo=tpe.suggest,
                                     max_evals=5,
                                     trials=Trials(),
                                     notebook_name='NeuralAnalysis')



>>> Imports:
#coding=utf-8

try:
    import numpy as np
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    from sklearn.preprocessing import StandardScaler
except:
    pass

try:
    from tensorflow.keras.models import Sequential
except:
    pass

try:
    from tensorflow.keras.layers import Dense
except:
    pass

try:
    from keras import backend
except:
    pass

try:
    from keras import backend
except:
    pass

try:
    from keras import backend as K
except:
    pass

try:
    from keras import losses
except:
    pass

try:
    from keras.callbacks import EarlyStopping
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    from sklearn.preprocessing import StandardScaler
except:
    pass

try:
    from keras import backend
except:
    pass

try:
 

TypeError: 'function' object is not subscriptable

In [ ]:
pip install networkx==1.11

In [ ]:

# if __name__=='__main__':
#     best_run, best_model= optim.minimize(model=create_model,
#                                         data=data,
#                                         algo=tpe.suggest,
#                                         max_evals=5,
#                                         trials=Trials())
#     x_train, y_train, x_test, y_test= data()
#     print('Evaluation of best performing model:')
#     print(best_model.evaluate(x_test,y_test))
#     print('Best performing model chosen hyper-parameters')
#     print(best_run)
    

#     score= model.evaluate(x_test,y_test, verbose=0)
#     accuracy= score[1]
#     return {'loss': -accuracy, 'status': STATUS_OK, 'model': model}